In [1]:
from pandas import read_csv
from pandas import datetime
from pandas import DataFrame
from matplotlib import pyplot
import sys
import pandas as pd
import numpy as np
import copy
import os
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import time
import math

In [13]:
import csv

curr_dir = os.getcwd()
dataset = pd.read_csv('CT15Mn-150818_101018/dados_camara.csv')#dados mais recentes
#tomtom = pd.read_csv("\\Users\\ASUS\\Documents\\IST\\5ºAno\\tomtom_data.csv")
#dataset = pd.read_csv("\\Users\\ASUS\\Documents\\IST\\5ºAno\\periodic_data.csv") #dados periodicos gerados automaticamente
#dataset = pd.read_csv("\\Users\\ASUS\\Documents\\IST\\5ºAno\\dados_old.csv") #dados mais antigos
dataset['unique_id'] = dataset.Zona.astype(str) + '_' + dataset.ID_Espira.astype(str)
dataset['unique_id'] = dataset['unique_id'].str.lower()
dataset = dataset[dataset["unique_id"] == "4_ct4"]
dataset = dataset.drop(columns=["unique_id"])

dataset['Data'] = pd.to_datetime(dataset['Data'], infer_datetime_format=True)
#print(dataset["Data"])

############# SHOULD BE INPUT FROM USER ###############

start_date = np.datetime64('2018-08-20')
end_date = np.datetime64('2018-08-24')

#######################################################

test = dataset[(dataset["Data"]>=start_date) & (dataset["Data"]<=end_date)]
#print(test.head())
test = test.values

############## GETTING ALL THE TIME INTERVALS BETWEEN THE SELECTED DATES ########################
start_time = datetime.strptime('2018-08-20', '%Y-%m-%d')
end_time = datetime.strptime('2018-08-25', '%Y-%m-%d')
def datetime_range(start, end, delta):
    current = start
    while current < end:
        yield current
        current += delta
hours = [dt.strftime('%Y-%m-%d %H:%M:%S') for dt in datetime_range(start_time,end_time, timedelta(minutes=15))]

############# CREATING DATASET ########################
def create_data(filename,dataset, hours):    
    #o = open("arima_sem_data.csv", "w", newline='')
    #f = open("arima_data.csv","w", newline='')
    f = open(str(filename)+".csv","w", newline='')
    out_file =  csv.writer(f, delimiter=',',quoting=csv.QUOTE_ALL) #csv.QUOTE_ALL)
    out_file.writerow(["Date/Step", "Count"])

    #new_file =  csv.writer(o, delimiter=',',quoting=csv.QUOTE_ALL) #csv.QUOTE_ALL)
    #new_file.writerow(["Date/Step", "Count"])

    lazy_hack = 0 
    lol = 0
    for row in range(len(dataset)):
        step = 1
        curr_values = dataset[row][4:]
        for value in range(len(curr_values)):
            out_file.writerow([hours[lazy_hack],curr_values[value]])
            lazy_hack+=1
            #new_file.writerow([lazy_hack, curr_values[value]])
            lol+=1
            f.flush()
            #o.flush()
    f.close()
    #o.close()
create_data("arima_data", test, hours)
#create_data("arima_sem_data", test, hours)

########################################################


######## TEST SET FIXME!!!! ####################################
test_date = np.datetime64('2018-08-27')
#end_test_date = np.datetime64('2018-08-28')
start_time = datetime.strptime('2018-08-27', '%Y-%m-%d')
end_time = datetime.strptime('2018-08-28', '%Y-%m-%d')
test_set = dataset[(dataset["Data"]==test_date)] # & (dataset["Data"]<=end_date)]
test_set = test_set.values

hours_set = [dt.strftime('%Y-%m-%d %H:%M:%S') for dt in datetime_range(start_time,end_time, timedelta(minutes=15))]
create_data("test_data", test_set, hours_set)


In [7]:
import plotly.plotly as ply
import cufflinks as cf
########### READING DATASET #####################
#arima_data2 = pd.read_csv("\\Users\\ASUS\\Documents\\IST\\5ºAno\\arima_sem_data.csv")#dados mais recentes
#print(arima_data.head())
def parsers(x):
    return datetime.strptime(x,'%Y-%m-%d %H:%M:%S')

#arima_data = read_csv("\\Users\\ASUS\\Documents\\IST\\5ºAno\\arima_data.csv",header=0, parse_dates=[0], index_col=0, squeeze=True, date_parser=parsers)

#print(arima_data.head())
#arima_data2.iplot(title="Traffic flow over a period of 5 days")

In [5]:
dataset = pd.read_sas("15minute_ByLaneData/md2004_15min_2a.sas7bdat") #md2004_15min_4b.sas7bdat
data_np = dataset.values
print(data_np)
miss_values = 0
for i in range(len(data_np)):
    if (math.isnan(data_np[i][1])):
        miss_values +=1
        past_values = len(data_np[:i])
        if past_values >= 4:
            temp = data_np[i-4:i]
            temp_2 = temp[:,1]
            avg = np.mean(temp_2)
            data_np[i][1] = avg
            #print("Changing Nan to :",avg)
        else:
            temp = data_np[i-past_values:i][1]
            temp_2 = temp[:,1]
            avg = np.mean(temp_2)
            data_np[i][1] = avg
            #print("(ELSE) Changing Nan to :",avg)
print(miss_values)
print(len(data_np))
df_data = pd.DataFrame(data_np)
df_data.columns = ['Data', 'Count']

df_data.to_csv("freeway_data1" + ".csv", sep=',', index=False)

#dataset.to_csv("freeway_data1.csv",sep = ",", index=False)
#dataset['Data'] = pd.to_datetime(dataset['Data'], infer_datetime_format=True)

            timestamp        flow
0 2004-01-01 00:00:00  201.333333
1 2004-01-01 00:15:00  157.333333
2 2004-01-01 00:30:00  300.000000
3 2004-01-01 00:45:00  356.000000
4 2004-01-01 01:00:00  440.000000
[[Timestamp('2004-01-01 00:00:00') 201.33333333333334]
 [Timestamp('2004-01-01 00:15:00') 157.33333333333334]
 [Timestamp('2004-01-01 00:30:00') 300.0]
 ...
 [Timestamp('2004-05-05 23:15:00') 533.3333333333334]
 [Timestamp('2004-05-05 23:30:00') 482.6666666666667]
 [Timestamp('2004-05-05 23:45:00') 469.3333333333333]]
622
12096


In [64]:
def create_dataset(dataset, look_back=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back-1):
        a = dataset[i:(i+look_back), 0]
        dataX.append(a)
        dataY.append(dataset[i + look_back, 0])
    return np.array(dataX), np.array(dataY)
# load the dataset
dataframe = pd.read_csv('freeway_data1.csv', usecols=[1], engine='python')
dataset = dataframe.values
dataset = dataset.astype('float32')

# split into train and test sets
train_size = int(len(dataset) * 0.67)
test_size = len(dataset) - train_size
train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]
print(len(train), len(test))
trainX, trainY = create_dataset(train,3)
testX, testY =create_dataset(test,3)
print(trainX)

8104 3992
[[201.33333 157.33333 300.     ]
 [157.33333 300.      356.     ]
 [300.      356.      440.     ]
 ...
 [714.6667  738.6667  802.6667 ]
 [738.6667  802.6667  748.     ]
 [802.6667  748.      770.6667 ]]


In [7]:
########## EXPERIMENTING WITH ARIMA #############
from plotly.plotly import plot_mpl
from statsmodels.tsa.seasonal import seasonal_decompose
from pyramid.arima import auto_arima
#arima_data["Count"] = pd.to_numeric(arima_data["Count"])
#print(arima_data["Date/Step"])
#x = arima_data.values
#print(x[:,0,None])
#arima_data.index = x[:,0]
#arima_data.index = pd.to_datetime(arima_data.index)
#print(arima_data.head())
#sys.exit()
result = seasonal_decompose(arima_data, model='multiplicative',freq=5)
fig = result.plot()
fig.show()


############### FINDING THE BEST PARAMETERS FOR P,Q,D


stepwise_model = auto_arima(data, start_p=1, start_q=1,
                           max_p=3, max_q=3, m=12,
                           start_P=0, seasonal=True,
                           d=1, D=1, trace=True,
                           error_action='ignore',  
                           suppress_warnings=True, 
                           stepwise=True)
print(stepwise_model.aic())

sys.exit()



model = ARIMA(arima_data, order=(5,1,0))
model_fit = model.fit(disp=0)
print(model_fit.summary())
residuals = DataFrame(model_fit.resid)
residuals.plot()
pyplot.show()
residuals.plot(kind='kde')
pyplot.show()
print(residuals.describe())




ModuleNotFoundError: No module named 'pyramid'